# Error propagation

## Tasks

a) Propagate uncertainties for the following expressions using [SymPy](https://www.sympy.org) following the examples for [uncorrelated variables](https://nbviewer.jupyter.org/urls/www.physi.uni-heidelberg.de/Einrichtungen/FP/Datenanalyse/FP_Gaussian_error_propagation.ipynb?flush_cache=false) and [correlated variables](https://nbviewer.jupyter.org/urls/www.physi.uni-heidelberg.de/Einrichtungen/FP/Datenanalyse/FP_Gaussian_error_propagation_corr.ipynb?flush_cache=false) from the FP web page.

i) Find expressions for the absolute uncertainty $\sigma_z$ for $z = x + y$ and $z = x - y$ 

ii) Find expressions for the relative uncertainty $\sigma_z / z$ for $z = x \cdot y, \; z = x / y$ and $z = x^n y^n$

iii) The acceleration of gravity with a simple pendulum is given by the following formula:
$$g = 4  \pi^2 \frac{L}{T^2}$$
The relevant variables are the length $L$ of the pendulum and the period $T$ with the corresponding errors $\sigma_L$ and $\sigma_T$.

b) The radius $r$ and the height $h$ of a cylinder have been measured to $r = 2$ cm and $h = 3$ cm. The uncertainty for both measurements is $\sigma = 0.05$ cm. Determine the volume of the cylinder and its uncertainty assuming (i) that both measurements are uncorrelated and (ii) that both measurements are fully correlated.

c) The scattering angle and the radial distance of a certain particle can be determined from a position measurement $(x,y)$ 
$$r = \sqrt{x^2 + y^2}, \quad \theta = \mathrm{atan2}(y, x)$$
You find more on the [atan2](https://en.wikipedia.org/wiki/Atan2) function on wikipedia. The position ($x$,$y$) is measured with the corresponding uncertainties $\sigma_x$ and $\sigma_y$. Write a python function that returns the covariance matrix $U$ of $r$ and $\theta$ for a given covariance matrix $V$ of $x$ and $y$. Determine $U$ under the assumption that $x$ and $y$ are uncorrelated. Hint: The formulas you need can be found in the script.


### Solution

In [16]:
import sympy as sp
import numpy as np
from IPython.display import display, Latex

def eprop_cor(f, data, covariance):
    S = sp.sympify("0")
    for i in range(len(data)):
        for j in range(len(data)):
            S += sp.diff(f, data[i]) * sp.diff(f, data[j]) * covariance[i][j]
    return sp.sqrt(S).simplify()

def sympy_equation(variable_latex, sympy_formula):
    display(Latex(f"${variable_latex} = {sp.latex(sympy_formula)}$"))
    
def display_eprop(var, var_eq, err, err_eq):
    rhos = [ 1, 0,-1]
    string = f"${var} = {sp.latex(var_eq)} \Rightarrow {err} = {sp.latex(err_eq)}"+r"=\left\{\begin{array}{ll}"
    for i in range(len(rhos)):
        string += f"{sp.latex(err_eq.subs([(rho, rhos[i])]).factor())}"+r", & \rho="+f"{rhos[i]}"
        if i+1 < len(rhos):
            string += r"\\ "
        else:
            string += r"\end{array}\right.$"
    display(Latex(string))
    
display(Latex(r"""For generally correlated error propagation, a function "eprop_cor" is defined.
It takes the one-dimensional function `f`, the parameters `data`, the covariance matrix `C` and returns the
propagated error: $$\sigma_f = \sqrt{\sum_{i, j=1}^{n} \frac{\delta f}{\delta x_i} 
\frac{\delta f}{\delta x_j} \cdot C_{ij}}$$ The helper-function `sympy_equation` facilitates the 
display of equations hereafter."""))

<IPython.core.display.Latex object>

In [79]:
x, y, rho = sp.symbols("x, y, rho", real=True)
sigma_x, sigma_y = sp.symbols("sigma_x, sigma_y", real=True, positive=True)
covariance_matrix_LT = [[sigma_x**2, rho * sigma_x * sigma_y], [rho * sigma_x * sigma_y, sigma_y**2]]

display(Latex("a)"))
display(Latex(r"""i) To calculate the propagated uncertainty of `z` for independent `x` and `y`,
the equation `z`, the measurements `x` and `y`, and the respective errors `\sigma_x` and `\sigma_y`
 are passed to the function eprop_cor, yielding the following equations:"""))
z = x + y
sigma_z = eprop_cor(z, [x, y], covariance_matrix_LT)
display_eprop("z", z, "\sigma_z", sigma_z)
z = x - y
sigma_z = eprop_cor(z, [x, y], covariance_matrix_LT)
display_eprop("z", z, "\sigma_z", sigma_z)


display(Latex("ii)"))
z = x * y
sigma_z = eprop_cor(z, [x, y], covariance_matrix_LT)
display_eprop("z", z, r"\frac{\sigma_z}{z}", sigma_z / z)
z = x / y
sigma_z = eprop_cor(z, [x, y], covariance_matrix_LT)
display_eprop("z", z, r"\frac{\sigma_z}{z}", sigma_z / z)

display(Latex(r"""For the product `z = x^ny^n` we introduce the new variable `n`, 
yielding the following error propagation:"""))
n, sigma_n = sp.symbols("n, sigma_n", real=True, positive=True)
z = x**n * y**n
sigma_z = eprop_cor(z, [x, y], covariance_matrix_LT)
display_eprop("z", z, r"\frac{\sigma_z}{z}", sigma_z / z)
display(Latex(r"For `n=1`, this yields the formula for simple multiplication as seen above:"))
display_eprop("z", z.subs([(n, 1)]), r"\frac{\sigma_z}{z}", (sigma_z / z).subs([(n, 1)]))


display(Latex("iii)"))
L, T, sigma_L, sigma_T = sp.symbols("L, T, sigma_L, sigma_T", real=True, positive=True)
covariance_matrix_LT = [[sigma_L**2, rho * sigma_L * sigma_T], [rho * sigma_L * sigma_T, sigma_T**2]]
rho = sp.symbols("rho", real=True)
g = 4 * sp.pi**2 * L / T**2
sigma_g = eprop_cor(g, [L, T], covariance_matrix_LT)
display(Latex("For the gravitational acceleration, it holds that"))
display_eprop("g", g, "\sigma_g", sigma_g)
display(Latex("and"))
display_eprop("g", g, r"\frac{\sigma_g}{g}", sigma_g / g)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [8]:
r, sigma_r, h, sigma_h = sp.symbols("r, sigma_r, h, sigma_h", real=True, positive=True)
rho = sp.symbols("rho", real=True)
r_val, sigma_r_val = 2, 5e-2
h_val, sigma_h_val = 3, 5e-2

V = sp.pi * r**2 * h
V_val = V.subs([(r, r_val), (h, h_val)]).evalf()
covariance_matrix = [[sigma_r**2, rho * sigma_r * sigma_h], [rho * sigma_h * sigma_r, sigma_h**2]]
sigma_V = eprop_cor(V, [r, h], covariance_matrix)

display(Latex("b)"))
display(Latex(r"""The volume `V` of a cylinder with radius `r \pm \sigma_r` and height `h \pm \sigma_h` yields, in general, the following error is thus calculated:"""))
display_eprop("V", V, r"\sigma_V", sigma_V)
for rho_val in [1, 0, -1]:
    sigma_V_val = sigma_V.subs([(r, r_val), (sigma_r, sigma_r_val), (h, h_val), (sigma_h, sigma_h_val), (rho, rho_val)]).evalf()
    display(Latex(r"For correlation `\rho = "+f"{rho_val}"+f"`, the result is `V = ({V_val:.1f} \pm {sigma_V_val:.1f}) " + "\ cm^3.`"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [9]:
def eprop_cor2d(y, x, covariance_matrix):
    n, m = len(x), len(y)
    E = [[sp.sympify("0") for i in range(m)] for i in range(m)]
    for k in range(m):
        for l in range(m):
            for i in range(n):
                for j in range(n):
                    E[k][l] += sp.diff(y[k], x[i]) * sp.diff(y[l], x[j]) * covariance_matrix[i][j]
            E[k][l] = E[k][l].simplify()
    return E

x, sigma_x, y, sigma_y = sp.symbols("x, sigma_x, y, sigma_y", real=True)
r, theta = sp.sqrt(x**2 + y**2), sp.atan2(y, x)
covariance_matrix_x_y = [[sigma_x**2, rho * sigma_x * sigma_y], [rho * sigma_y * sigma_x, sigma_y**2]]
covariance_matrix_r_theta = eprop_cor2d((r, theta), (x, y), covariance_matrix_x_y)
covariance_matrix_r_theta_ = sp.Matrix(covariance_matrix_r_theta).subs([(rho, 0)])

display(Latex("c)"))
display(Latex(r"For multidimensional error propagation we define the function eprop_cor2d. Here `\vec{y}(\vec{x})` depends on the parameters `\vec{x} = (x_1, x_2, ..., x_n)`."))
display(Latex(r"To calculate the new covariance matrix `U` of `r` and `\theta` we first write:"))
sympy_equation("r", r)
sympy_equation(r"\theta", theta)
display(Latex(r"and form the covariance matrix of `x` and `y`:"))
display(Latex(f"$V = {sp.latex(sp.Matrix(covariance_matrix_x_y))}$"))
display(Latex(r"The new covariance matrix `U` is now calculated analytically and used for the correlation `\rho = 0`:"))
display(Latex(f"$U = {sp.latex(sp.Matrix(covariance_matrix_r_theta))} = {sp.latex(sp.simplify(covariance_matrix_r_theta_))}$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>